In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from string import punctuation
import spacy
import re
from IPython.core.display import display, HTML

In [ ]:
df = pd.read_json('calm_records.json')

In [ ]:
soup = BeautifulSoup(df.loc[269057]['AdminHistory'][0], 'html.parser')
basic_text = soup.get_text()

In [ ]:
print(basic_text)

In [ ]:
nlp = spacy.load('en')
doc = nlp(basic_text)

for ent in doc.ents:
    if ent.label_ != 'DATE' and len(ent.text.split()) > 1:
        words = ent.text.lower().split()
        words = [word.replace("'s", '') for word in words]
        words = [word.translate(str.maketrans('', '', punctuation)) 
                 for word in words]

        print('https://en.wikipedia.org/w/index.php?search=' + 
              '+'.join(words))

In [ ]:
nlp = spacy.load('en')
doc = nlp(basic_text)

links = {}

for ent in doc.ents:
    if ent.label_ != 'DATE' and len(ent.text.split()) > 1:
        words = ent.text.lower().split()
        words = [word.translate(str.maketrans('', '', punctuation)) 
                 for word in words]
        url = ('https://en.wikipedia.org/w/index.php?search=' + 
                '+'.join(words))
        link = '<a href="{}">{}</a>'.format(url, ent.text.strip())
        links[ent.text.strip()] = link

In [ ]:
pattern = re.compile(r'\b(' + '|'.join(links.keys()) + r')\b')
result = pattern.sub(lambda x: links[x.group()], str(soup))

display(HTML(result))